In [ ]:
%%html
<!--- 
The camera widget must be visible on some devices for the video capture to work.
--->

<video id="video" autoplay style="width:640px; height:360px;"></video> 

<!---
<video id="videofile" controls autoplay loop style="width:320px; height:240px;" crossOrigin="Anonymous"; src="https://is3l.isr.uc.pt/~pm/CGRA/marcador65.mp4"></video>
--->

<canvas id="myCanvas" width="640" height="360" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>

In [ ]:
%%html


<!--- all the access to the auxiliary files must use https instead of http --->
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgratexture.js"></script>
<script src='https://git.io/glm-js.min.js'></script>

<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/js-aruco/cv.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/js-aruco/aruco.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/js-aruco/svd.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/js-aruco/posit1.js"></script>
<script src="file:///C:/Users/telmo/OneDrive/Ambiente%20de%20Trabalho/Faculdade/Material%20de%20Apoio/5%C2%BAano/Computa%C3%A7%C3%A3o%20Gr%C3%A1fica%20e%20Realidade%20Aumentada/lab3/calib_integrated_camera.json">

<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main() {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main() {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>


<script id="my-vertex-shaderC" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;
uniform vec3 un_Color;
varying  vec3 ex_Color;

void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = un_Color;
}
</script>


<script id="my-vertex-shaderT" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
attribute vec2 in_texcoords;
uniform mat4 MVP;

varying  vec3 ex_Color;
varying  vec2 vTextureCoord;

void main() {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);
    vTextureCoord = in_texcoords;
    ex_Color = in_Color;
}
</script>

<script id="my-fragment-shaderT" type="x-shader/x-fragment">
precision mediump float;
varying  vec2 vTextureCoord;
varying  vec3 ex_Color;

uniform sampler2D uSampler;

void main() {
     gl_FragColor = texture2D(uSampler, vTextureCoord);
}
</script>




In [ ]:
%%html
<script>
class triangle extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 6;
        var vertices =
            [ -0.95, -0.95, 0.9,
              0.95, -0.95, 0.9,
              -0.95, 0.95, 0.9,
              0.95, 0.95, 0.9,
              0.95, -0.95, 0.9,
              -0.95, 0.95, 0.9
            ]; 
        
        var colors = [
            1.0, 0.0, 0.0, 
            0.0, 1.0, 0.0,
            0.0, 0.0, 1.0,
            1.0, 1.0, 0.0,
            1.0, 0.0, 0.0,
            0.0, 1.0, 0.0
        ];
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
    }
}

class triangleT extends triangle{
    
    constructor(glcontext){
        super(glcontext);
            var texcoords = [
                0.0, 0.0,
                1.0, 0.0,
                0.0, 1.0,
                1.0, 1.0,
                1.0,0.0,
                0.0,1.0];
        
            this.texcoordbuffer = this.gl.createBuffer();
            this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordbuffer);    
            this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(texcoords), this.gl.STATIC_DRAW);
    }
        
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
       
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }    
}
</script>

In [ ]:
%%html
<script>
class esferatext extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 2160;
        var centro = [0.0, 0.0, 0.0];
        var disco = [];
        var colors = [];
        var actualvertices = [];
        const raioConstante = 1.0;
        //var ind;
        
        var r = [1.0, 0.0, 0.0];
        var g = [0.0, 1.0, 0.0];
        var b = [0.0, 0.0, 1.0];
        
        var textcoord = [];
               
        for (var ind=0; ind<=10; ind++){
            disco[ind] = [];
            
            var raio = raioConstante * Math.cos((Math.PI/10*ind) - Math.PI/2);
            
            for (var j=-Math.PI/2.0; j<=3.0*Math.PI; j+=1.0*Math.PI/10.0)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),Math.sin((Math.PI/10*ind) - Math.PI/2));
    
        }
        
        var offsety = 1.0/(disco.length);
        var offset2 = 0.0;
        for (var i=0; i<disco.length-1 ; i++){
            
            var offset1 = 0.0;
            var offsetx = 1.0/20; 
            
            for (var j=0; j<disco[0].length; j+=3){
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);  // 1 ponto
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);  // 1 ponto seguinte mesmo disco
                colors = colors.concat(r,g,b);
                
                textcoord=textcoord.concat([offset1,(offset2+offsety)]); //ponto disco2
                textcoord=textcoord.concat([offset1,offset2]); //ponto disco 1
                textcoord=textcoord.concat([(offset1+offsetx),offset2]); //ponto2 disco 1
   
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte mesmo disco
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);  // 1 ponto do disco abaixo
                colors = colors.concat(r,g,b);
                
                textcoord=textcoord.concat([offset1,(offset2+offsety)]); //ponto disco2
                textcoord=textcoord.concat([(offset1+offsetx),(offset2+offsety)]); //ponto2 disco2
                textcoord=textcoord.concat([(offset1+offsetx),offset2]); //ponto 2 disco1
                
                offset1=offset1+offsetx;
           }
            offset2=offset2+offsety;
        }     
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.texturebuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texturebuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(textcoord), this.gl.STATIC_DRAW);
    }
}

class esferaT extends esferatext{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texturebuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.textureLocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

In [ ]:
%%html
<script>
class cone extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        
        this.numvertices = 60;//60
        var disco = [];
        var actualvertices = [];
        var ind;
        
        var r = [1.0, 0.0, 0.0];
        var g = [0.0, 1.0, 0.0];
        var b = [0.0, 0.0, 1.0];
                 
        var colors = [];
               
        var numlados = this.numvertices /6;
        var offsetx= 1/numlados;
        var textcoord = [];
        
               
        for (var i=-1.0; i<=-1.0; i+=1){
            console.log(i);
            ind = 0;
            var raio = 1.0;
            disco[ind] = [];
            for (var j=-Math.PI/2.0; j<=3*Math.PI/2.0; j+=2.0*Math.PI/10)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),i);
        }
        
        //# Unir pontos dos discos ao ponto extremo
        var offset=0;
        for (var j=0; j<30; j+=3){
                actualvertices = actualvertices.concat(disco[0][j],disco[0][j+1],disco[0][j+2]);  
                actualvertices = actualvertices.concat(0,0,1); 
                actualvertices = actualvertices.concat(disco[0][j+3],disco[0][j+4],disco[0][j+5]);
                colors = colors.concat(r,g,b);
            
                //text coords (pontos externos do disco ao apex,(topo))
                textcoord = textcoord.concat([offset, 0.0]);
                textcoord = textcoord.concat([offsetx, 1.0]);
                textcoord = textcoord.concat([offset + offsetx, 0.0]);
                offset=offsetx+offset;
        }
        
        //# Unir pontos dos discos ao centro
        for (var j=0; j<30; j+=3){
                actualvertices = actualvertices.concat(disco[0][j],disco[0][j+1],disco[0][j+2]);  
                actualvertices = actualvertices.concat(0,0,-1); 
                actualvertices = actualvertices.concat(disco[0][j+3],disco[0][j+4],disco[0][j+5]);
                colors = colors.concat(r,g,b);
            
                textcoord= textcoord.concat([0.00,0.00]);//ponto de cima 1
                textcoord= textcoord.concat([0.5,0.5]);//ponto de cima 2
                textcoord= textcoord.concat([1.00,0.00]);// ponto de baixo 2
        }
               
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(textcoord), this.gl.STATIC_DRAW);
            
        
    }
}


class coneT extends cone{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

In [ ]:
%%html
<script>
class cube extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.vColors = [];
        this.vPositions = [];
        this.texCoords = [];
        
        this.Index=0;
        var r=1.0;
        var g=0.5;
        var b= 0.6;
        this.positions =
            [ glm.vec3(-1.0, -1.0, 1.0),
              glm.vec3(-1.0, 1.0, 1.0),
              glm.vec3(1.0, 1.0, 1.0),
              glm.vec3(1.0, -1.0, 1.0),
              glm.vec3(-1.0, -1.0, -1.0),
              glm.vec3(-1.0, 1.0, -1.0),
              glm.vec3(1.0, 1.0, -1.0),
              glm.vec3(1.0, -1.0, -1.0)]; 
        
        this.colors = [
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b),
              glm.vec3(r, g, b)];
        
        this.tcoords = [
            //Front
            glm.vec2(0.25,0.6666),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.25,0.6666),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.50,0.6666),
            
            //Botton
            glm.vec2(0.50,0.6666),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.75,0.3333),
            glm.vec2(0.50,0.6666),
            glm.vec2(0.75,0.3333),
            glm.vec2(0.75,0.6666),
            
            //Back
            glm.vec2(0.75,0.6666),
            glm.vec2(0.75,0.3333),
            glm.vec2(1.00,0.3333),
            glm.vec2(0.75,0.6666),
            glm.vec2(1.00,0.3333),
            glm.vec2(1.00,0.6666),
            
            //left
            glm.vec2(0.50,0.3333),
            glm.vec2(0.50,0.0000),
            glm.vec2(0.75,0.0000),
            glm.vec2(0.50,0.3333),
            glm.vec2(0.75,0.0000),
            glm.vec2(0.75,0.3333),
            
            //rigth
            glm.vec2(0.50,1.0000),
            glm.vec2(0.50,0.6666),
            glm.vec2(0.75,0.6666),
            glm.vec2(0.50,1.0000),
            glm.vec2(0.75,0.6666),
            glm.vec2(0.75,1.0000),
            
            //Top
            glm.vec2(0.00,0.6666),
            glm.vec2(0.00,0.3333),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.00,0.6666),
            glm.vec2(0.25,0.3333),
            glm.vec2(0.25,0.6666)
        ];
        
      
        this.quad( 1, 0, 3, 2, 0);
        this.quad( 2, 3, 7, 6, 6);
        this.quad( 3, 0, 4, 7, 12);
        this.quad( 6, 5, 1, 2, 18);
        this.quad( 4, 5, 6, 7, 24);
        this.quad( 5, 4, 0, 1, 30);
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        var a = new Float32Array(this.vPositions);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        a = new Float32Array(this.vColors);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        
        this.texcoordsbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texcoordsbuffer);  
        a=new Float32Array(this.texCoords);
        this.gl.bufferData(this.gl.ARRAY_BUFFER, a, this.gl.STATIC_DRAW);
        //console.log("texc=" + this.texCoords+ "  a= "+ a + " len= "+a.length)
        
        this.numvertices = 36;
    }
    
     quad(a, b, c, d, side)
    {
  
        this.vColors = [].concat(this.vColors,this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side].array);

        this.vColors = [].concat(this.vColors, this.colors[b].array); 
        this.vPositions = [].concat(this.vPositions,this.positions[b].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+1].array);
             
        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side+2].array);

        this.vColors = [].concat(this.vColors, this.colors[a].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[a].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+3].array);

        this.vColors = [].concat(this.vColors, this.colors[c].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[c].array); 
        this.texCoords = [].concat(this.texCoords,this.tcoords[side+4].array);

        this.vColors = [].concat(this.vColors,this.colors[d].array); 
        this.vPositions = [].concat(this.vPositions, this.positions[d].array); 
        this.texCoords = [].concat(this.texCoords, this.tcoords[side+5].array);

    }
    
}

class cubeT extends cube{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texcoordsbuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.texturelocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

In [ ]:
%%html
<script>
class CompoundObject extends CGRAobject{
    array=[];
    obj;
    
    //metodo para inserir objeto
    insertObject(objeto, transformation = glm.mat4(1.0)){
        this.obj = objeto;
        this.obj.setModelTransformation(transformation);
        this.array = this.array.concat(this.obj);
    }
  
    setShader(shader){
        for (var i=0; i<this.array.length; i++)
            this.array[i].shaderprog = shader;
    }
   
    drawit(V,P,parent=glm.mat4(1.0)){
        var trans;
        for(var i=0;i<this.array.length; i++){
            trans = parent['*'](this.array[i].modelMat);
            var aux = this.array[i].modelMat;
            this.array[i].modelMat = trans;
            this.array[i].drawit(V,P);
            this.array[i].modelMat = aux;
        }
    }
  
};
</script>

In [ ]:
%%html
<script>
class cilindro extends CGRAobject{
    constructor(glcontext){
        super(glcontext); // initialize the parent class
        this.numvertices = 180;
        var disco = [];
        var actualvertices = [];
        var colors= [];
        var ind;
        
        var r = [1.0, 0.0, 0.0];
        var g = [0.0, 1.0, 0.0];
        var b = [0.0, 0.0, 1.0];
        
        var numlados = this.numvertices/12;
        var offsetx= 1.0/numlados;
        var textcoord = [];
        
        for (var i=-1.0; i<=1.0; i+=2){
            if(i == -1.0)
                ind = 0.0;
            else 
                ind = 1.0
            
            var raio = 1.0;
            disco[ind] = [];
            for (var j=-Math.PI/2.0; j<=3*Math.PI/2.0; j+=2.0*Math.PI/10)
                disco[ind] = disco[ind].concat(raio*Math.cos(j),raio*Math.sin(j),i);
            
        }
        // Unir pontos entre discos
        var offset=0;
        for (var i=0; i<1 ; i++){
            //cada ciclo desenha uma face
            for (var j=0; j<30; j+=3){
                

                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto do disco acima
                actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto seguinte
                colors = colors.concat(r,g,b);
                
                textcoord= textcoord.concat([offset,1.00]);//ponto de cima
                textcoord= textcoord.concat([offset,0.00]);//ponto
                textcoord= textcoord.concat([(offset+offsetx),0.00]);//ponto seguinte
                
                actualvertices = actualvertices.concat(disco[i+1][j],disco[i+1][j+1],disco[i+1][j+2]); // 1 ponto
                actualvertices = actualvertices.concat(disco[i+1][j+3],disco[i+1][j+4],disco[i+1][j+5]); // 1 ponto seguinte
                actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]); // 1 ponto do disco acima
                colors = colors.concat(r,g,b);
                
                textcoord= textcoord.concat([offset,1.00]);//ponto de cima 1
                textcoord= textcoord.concat([(offset+offsetx),1.00]);//ponto de cima 2
                textcoord= textcoord.concat([(offset+offsetx),0.00]);// ponto de baixo 2
                
                //console.log("Debug:coord textura:");
                //console.log(textcoord);
                
                offset=offset+offsetx;
            }
            
        }

        // Unir pontos dos discos ao centro
        for (var i=0; i<=1 ; i++){
            for (var j=0; j<30; j+=3){
                if(i==0){
                    actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);  
                    actualvertices = actualvertices.concat(0,0,-1); 
                    actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);
                    colors = colors.concat(r,g,b);
                    
                    textcoord= textcoord.concat([0.00,0.00]);//ponto de cima 1
                    textcoord= textcoord.concat([0.5,0.5]);//ponto de cima 2
                    textcoord= textcoord.concat([1.00,0.00]);// ponto de baixo 2
                }
                if(i==1){
                    actualvertices = actualvertices.concat(disco[i][j],disco[i][j+1],disco[i][j+2]);  
                    actualvertices = actualvertices.concat(0,0,1); 
                    actualvertices = actualvertices.concat(disco[i][j+3],disco[i][j+4],disco[i][j+5]);
                    colors = colors.concat(r,g,b);
                    
                    textcoord= textcoord.concat([0.00,0.00]);//ponto de cima 1
                    textcoord= textcoord.concat([0.5,0.5]);//ponto de cima 2
                    textcoord= textcoord.concat([1.00,0.00]);// ponto de baixo 2
                }
                
            }
        }
        
        this.vertexbuffer=this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(actualvertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.texturebuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.texturebuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(textcoord), this.gl.STATIC_DRAW);
    }
}
class cilindroT extends cilindro{
    
    settexture(cgratex){
            this.textureid = cgratex.textureid;
    }
    
    drawit(viewMat, projectionMat){
        this.shaderprog.startUsing();
        this.texcoordsLocation = this.gl.getAttribLocation(this.shaderprog.shaderProgram,
                                                          "in_texcoords");
        
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER,this.texturebuffer);
        this.gl.vertexAttribPointer(this.texcoordsLocation, // Attribute location
                           2, // number of elements per attribute
                           this.gl.FLOAT,  // Type of elements
                           false,  // 
                           0, //2*Float32Array.BYTES_PER_ELEMENT, // size of a vertex in bytes 
                           0); // Offset from the begining of a single vertex to this attribute
        this.gl.enableVertexAttribArray(this.texcoordsLocation);
        this.gl.bindTexture(this.gl.TEXTURE_2D, this.textureid);
        this.textureLocation =  this.gl.getUniformLocation(this.shaderprog.shaderProgram, "uSampler");
        this.gl.uniform1i(this.textureLocation, 0);
        // the parent method does the rest
        super.drawit(viewMat,projectionMat);
    }  
}
</script>

In [ ]:
%%html

<pre id=testfield> </pre>
<pre id=markerfield></pre>

<script>
class myapp extends DEECapp{
    counter=0;
    initvidefile(){
        this.video = document.getElementById("videofile");
        this.back = document.createElement('canvas');
        this.backctx = this.back.getContext('2d');
    }
    initcamera(){
        this.video = document.getElementById("video");
        this.back = document.createElement('canvas');
        this.backctx = this.back.getContext('2d');
      if (navigator.mediaDevices === undefined) {
        navigator.mediaDevices = {};
      }
      
      if (navigator.mediaDevices.getUserMedia === undefined) {
        navigator.mediaDevices.getUserMedia = function(constraints) {
          var getUserMedia = navigator.webkitGetUserMedia || navigator.mozGetUserMedia;
          
          if (!getUserMedia) {
            return Promise.reject(new Error('getUserMedia is not implemented in this browser'));
          }

          return new Promise(function(resolve, reject) {
            getUserMedia.call(navigator, constraints, resolve, reject);
          });
        }
      }
      
      navigator.mediaDevices
        .getUserMedia({ video: true })
        .then(function(stream) {
          if ("srcObject" in video) {
            video.srcObject = stream;
          } else {
            video.src = window.URL.createObjectURL(stream);
          }
        })
        .catch(function(err) {
          console.log(err.name + ": " + err.message);
        }
      );
    }
    
    
    initialize(){

        if ( document.getElementById("videofile")){
            this.initivideofile();
        } 
        else {
            this.initcamera();
        }

        var fragsrcT = document.getElementById("my-fragment-shaderT").text;
        var vertsrcT = document.getElementById("my-vertex-shaderT").text;
        
        this.shaderprogT = new DEECshader(this.gl);
        this.shaderprogT.srcShaders(vertsrcT,fragsrcT);
              
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(0.30,0.30,0.30,1.0);
        
        //===============================================================
        
        // Projection and view matrices
        this.projM = glm.perspective(glm.radians(45),1,1,100);
        this.viewM = glm.lookAt(glm.vec3(0,0,0),glm.vec3(0,0,-1),glm.vec3(0,1,0))
               
                
        
        // Projection and view matrices
        var width=640;
        var heigth=360;
       
        var zfar=10.0;
        var znear=1.0;
        /*
        var aux1= (width-(2*603.3097174869463)+(2*0))/width;
        var aux2= (heigth-(2*403.99626103199)+(2*0))/heigth;
        */
        //#"y-up"
        /*
        this.viewM = [
            979.4949095680221, 0, 603.3097174869463,
            0, 980.0627651589923,403.99626103199,
            0, 0,  1
        ];
        */
        /*
        this.projM =[
            (2*979.4949095680221)/width, (-2*0)/width, aux1, 0,
            0, (-2*980.0627651589923)/heigth, aux2, 0,
            0, 0, (-zfar-znear)/(zfar-znear), (-2*zfar*znear)/(zfar-znear),
            0, 0, -1, 0
        ];
        */
        /*
        //#"y-down"
        this.projM =[
            (2*this.viewM[0])/width, (-2*0)/1280, aux1, 0,
            0, (2*980.0627651589923)/heigth, -aux2, 0, 
            0, 0, (-zfar-znear)/(zfar-znear), (-2*zfar*znear)/(zfar-znear),
            0, 0, -1, 0
        ];
        */
        
        //================================================================
        // Lets create two triangle objects
        
        this.mytriangle = new triangleT(this.gl);
        this.mytriangle.setShader(this.shaderprogT);
        
        this.texture3 = new CGRAtexture(this.gl);
        
        this.mytriangle.settexture(this.texture3);
        
        this.texture1 = new CGRAtexture(this.gl);
        this.texture1.load("le_texture.png")
        this.mycubo= new cubeT(this.gl);
        this.mycubo.settexture(this.texture1);
        this.mycubo.setShader(this.shaderprogT);
        
        this.textureCone = new CGRAtexture(this.gl);
        this.textureCone.load("treeleaf.png");
        this.mycone = new coneT(this.gl);
        this.mycone.settexture(this.textureCone);
        this.mycone.setShader(this.shaderprogT);
        
        this.textureEsfera = new CGRAtexture(this.gl);
        this.textureEsfera.load("Xmaspaper.jpg");
        this.myesfera = new esferaT(this.gl);
        this.myesfera.settexture(this.textureEsfera);
        this.myesfera.setShader(this.shaderprogT);
        
        
        this.textureCilindro = new CGRAtexture(this.gl);
        this.textureCilindro.load("trunk.png");
        this.mycilindro = new cilindroT(this.gl);
        this.mycilindro.settexture(this.textureCilindro);
        this.mycilindro.setShader(this.shaderprogT);
        
       
        this.test = new CompoundObject;
        
        var scale = glm.mat4(glm.mat3(0.3));
        var rotate = glm.toMat4(glm.angleAxis(glm.radians(-90),glm.vec3(1.0,0.0,0.0)));
        var translate = glm.translate(glm.vec3(0.0,0.5,0.0));
        var transformation = translate['*'](rotate['*'](scale));
        this.test.insertObject(this.mycone, transformation);
        
        var scale = glm.scale(glm.vec3(0.1,0.1,0.25));
        var rotate = glm.toMat4(glm.angleAxis(glm.radians(90),glm.vec3(1.0,0.0,0.0)));
        var translate = glm.translate(glm.vec3(0.0,0.05,0.0));
        var transformation = translate['*'](rotate['*'](scale));
        this.test.insertObject(this.mycilindro, transformation);
             
    
        // HERE IS the initalization of the Aruco detector
        this.detector = new AR.Detector();
        var modelsize = 35.0;
        var focallength = 640.0;
        // and of the pose estimator
        this.posit = new POS.Posit(modelsize,focallength);
    }
    
    calculateSize(srcSize, dstSize) {
    var srcRatio = srcSize.width / srcSize.height;
    var dstRatio = dstSize.width / dstSize.height;
    if (dstRatio > srcRatio) {
      return {
        width:  dstSize.height * srcRatio,
        height: dstSize.height
      };
    } else {
      return {
        width:  dstSize.width,
        height: dstSize.width / srcRatio
      };
    }
  }
    
    processvideo(){
        
       
        if (this.video.readyState === this.video.HAVE_ENOUGH_DATA) {
            var videoSize = { width: this.video.videoWidth, height: this.video.videoHeight };
            var canvasSize = { width: this.canvas.width, height: this.canvas.height };
            var renderSize = this.calculateSize(videoSize, canvasSize);
            var xOffset = (canvasSize.width - renderSize.width) / 2;
          
            this.back.width=this.video.videoWidth;
            this.back.height=this.video.videoHeight;
            this.backctx.drawImage(this.video,0,0,this.video.videoWidth,this.video.videoHeight);
            var imagedata = this.backctx.getImageData(0,0,this.video.videoWidth,this.video.videoHeight);
            
            this.texture3.update(imagedata,this.video.videoWidth,this.video.videoHeight);    
            //
            // Detect the markers 
            var markers = this.detector.detect(imagedata);
            
            var mText ="Markers detected = " + markers.length + "\n";
            if (markers.length > 0){
                for (var j = 0; j !== markers.length; ++ j){
                    mText += "Marker "+j + " ID= " + markers[j].id +"\n";
                    var corners = markers[j].corners;
                
                    for (var i=0; i !== corners.length; ++ i){
                        var corner = corners[(i + 1) % corners.length];
                        mText += "Corner " + i + " " + corner.x + " " + corner.y + "\n";
                   
                    }
                    
                    switch(markers[j].id){
                        case 0: this.flag = 0; break;
                        case 1: this.flag = 1; break;
                        case 2: this.flag = 2; break; 
                        case 3: this.flag = 3; break;
                        default: break;
                    }

                }
                var pose = this.posit.pose(corners);
                var translation = pose.bestTranslation;
                var rotation = pose.bestRotation;
                
                this.rot = rotation;
                
                //this.posx = translation[0]; //translation[0];
                //this.posy = translation[1]; //translation[1];
                //this.posz = translation[2]; //translation[2];
                               
                
                this.posx = ((translation[0] - translation[2]/2)/translation[2])*(3);
                this.posy = ((translation[1] -(0.80*translation[2])/2)/(0.80*translation[2]))*(-3);
                this.posz = (-40)/translation[2];
               
                //calcular norma á mao
                //var norma = Math.sqrt(Math.pow(this.posx,2.0) + Math.pow(this.posy,2.0) + Math.pow(this.posz, 2.0));
                
                //this.posx = (this.posx/norma);
                //this.posy = (this.posy/norma);
                //this.posz = (this.posz/norma);
                
                
                
                
                var txtText = "Rotation = \n";
                for (var l=0;l<3;l++){
                    testfield.innerText += "|";
                    for (var c=0;c<3;c++){

                        txtText += Math.round(rotation[l][c]*100)/100;
                        txtText += ", ";
                    }
                    txtText += "|\n";
                }
        
                // pose.bestRotation, pose.bestTranslation);
                txtText += "Translation = \n [" + Math.round(translation[0]*100)/100 + ", "+ Math.round(translation[1]*100)/100 +", "+ Math.round(translation[2]*100)/100 + " ]";
                txtText += "Translation calculada = \n [" + this.posx + ", "+this.posy +", "+ this.posz + " ]";
                testfield.innerText = txtText;
                markerfield.innerText=mText;
            
            }
        }      
    }

    
    render(){
        this.gl.clear(this.gl.COLOR_BUFFER_BIT | this.gl.DEPTH_BUFFER_BIT);
        this.processvideo();
        this.counter++;
               
        var scale = glm.mat4(glm.mat3(1.0));
        var rotate = glm.toMat4(glm.angleAxis(glm.radians(180),glm.vec3(1.0,0.0,0.0)));
        var translate = glm.translate(glm.vec3(0.0,0.0,1.0));
        var transformation = translate['*'](rotate['*'](scale));
        this.mytriangle.setModelTransformation(transformation);
        // define the model tranformation for the second
               
        // draw both triangles
        this.mytriangle.drawit();
        
               
        // define the model tranformation for the first
        switch(this.flag){
            case 0:
                console.log("cubo\n");
                var scale = glm.mat4(glm.mat3(this.posz));
                var rotate = glm.mat4(glm.mat3(this.rot));
           
                var translate = glm.translate(glm.vec3(this.posx, this.posy,0.0));
                var transformation = translate['*'](rotate['*'](scale));
            
                this.mycubo.setModelTransformation(transformation);
                this.mycubo.drawit();
                break;
            
            case 1:
                console.log("Cone\n");
                var scale = glm.mat4(glm.mat3(0.2));
                var rotate = glm.mat4(glm.mat3(this.rot));
            
                var translate = glm.translate(glm.vec3(this.posx,this.posy,0.0));
                var transformation = translate['*'](rotate['*'](scale));
            
                this.mycone.setModelTransformation(transformation);
                this.mycone.drawit();
                break;
            
            case 2:
                console.log("esfera\n");
                var scale = glm.mat4(glm.mat3(0.2));
                var rotate = glm.mat4(glm.mat3(this.rot));
            
                var translate = glm.translate(glm.vec3(this.posx,this.posy,0.0));
                var transformation = translate['*'](rotate['*'](scale));
            
                this.myesfera.setModelTransformation(transformation);
                this.myesfera.drawit();
                break;
            
            case 3: 
                console.log("compound object\n");
                var scale = glm.mat4(glm.mat3(0.2));
                var rotate = glm.mat4(glm.mat3(this.rot));
            
                var translate = glm.translate(glm.vec3(this.posx,this.posy,0.0));
                var transformation = translate['*'](rotate['*'](scale));
            
                this.test.setModelTransformation(transformation);
                this.test.drawit();//this.viewM, this.projM
                break;
            
            default:
                console.log("nothing\n");
                break;
                
        }
        
   
    }
}

var app = new myapp('myCanvas');

app.run();

</script>